In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from collections import defaultdict
from tqdm import tqdm
import sys
sys.path.append('..')
from utils import ising_data_builder, H5IsingDataset



# Red Neuronal Fully Connected para Clasificación del Modelo Ising

Este notebook entrena y evalúa una red neuronal fully connected para clasificar las fases del modelo Ising.



## Construcción del Dataset



In [ ]:
# Crear el dataset
data = ising_data_builder('../data/').h5_path
dataset = H5IsingDataset(data)



## División Train/Test



In [ ]:
n = len(dataset)
labels = np.asarray(dataset.y[:]) 

# Train / Test Split estratificado
train_idx, test_idx = train_test_split(
    np.arange(n),
    test_size=0.2,
    stratify=labels,
    random_state=4
)

# Subsets que no copian datos, solo crean vistas por índices
train_set = Subset(dataset, train_idx.tolist())
test_set = Subset(dataset, test_idx.tolist())

# DataLoaders -> genera los batches
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, num_workers=0)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False, num_workers=0)



## Definición del Modelo



In [ ]:
class IsingNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()

        # Topología de la red 
        self.model = nn.Sequential(
            nn.Flatten(),          # Aplana el tensor a (batch, N)
            nn.Linear(input_dim, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 200),
            nn.ReLU(),
            nn.Linear(200, 100),
            nn.ReLU(),
            nn.Linear(100, 50),
            nn.ReLU(),
            nn.Linear(50, 1)
            # Nota: No aplicamos Sigmoid aquí porque BCEWithLogitsLoss lo hace internamente
            # de forma más estable numéricamente
        )

    def forward(self, x):
        return self.model(x)

# Model, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = IsingNN(input_dim=100).to(device)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)



## Funciones de Entrenamiento y Evaluación



In [ ]:
# Entrenamiento
def train_epoch(model, loader):
    model.train()
    total_loss, total_correct, total = 0, 0, 0

    for X, y in loader:
        X, y = X.to(device), y.float().to(device)

        logits = model(X).squeeze()  # Forward y eliminar dimensión extra [batch, 1] -> [batch]
        loss = loss_fn(logits, y)    # Loss

        optimizer.zero_grad() # Limpiar gradientes
        loss.backward()       # Backward propagation
        optimizer.step()      # Update weights and biases

        total_loss += loss.item() * X.size(0) # Pérdida del batch

        preds = (torch.sigmoid(logits) >= 0.5).long()     # Logits a labels
        total_correct += (preds == y.long()).sum().item() # Contar correctos
        total += X.size(0) 
        
    # Pérdida promedio y accuracy
    return total_loss / total, total_correct / total 

# Evaluación
def eval_epoch(model, loader):
    model.eval()
    total_loss, total_correct, total = 0, 0, 0

    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.float().to(device)

            logits = model(X).squeeze()  # Eliminar dimensión extra [batch, 1] -> [batch]
            loss = loss_fn(logits, y)

            total_loss += loss.item() * X.size(0)
            preds = (torch.sigmoid(logits) >= 0.5).long()
            total_correct += (preds == y.long()).sum().item()
            total += X.size(0)

    return total_loss / total, total_correct / total



In [ ]:
# Loop de entrenamiento
for epoch in tqdm(range(1, 6), desc="Epochs"):
    train_loss, train_acc = train_epoch(model, train_loader)
    test_loss, test_acc = eval_epoch(model, test_loader)

    print(f"Epoch {epoch} | "
          f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | "
          f"Test Loss: {test_loss:.4f}, Acc: {test_acc:.4f}")



## Matriz de Confusión



In [ ]:
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for X, y in test_loader:
        X = X.to(device)

        logits = model(X)

        # Convertir a clases 0/1
        preds = (torch.sigmoid(logits) >= 0.5).long()

        # Guardar todo
        all_preds.append(preds.cpu())
        all_labels.append(y.long().cpu())

# Convertir listas de tensores a arrays numpy concatenados
all_preds = torch.cat(all_preds).numpy() 
all_labels = torch.cat(all_labels).numpy()

# Matriz de confusión
cm = confusion_matrix(all_labels, all_preds, normalize='true')

plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt=".2f", cmap="Blues")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("Matriz de Confusión")
plt.tight_layout()
plt.savefig('../img_results/confusion_matrix_nn.png', dpi=300, bbox_inches='tight')
plt.show()



## Accuracy por Temperatura



In [ ]:
# Análisis de accuracy por temperatura
import os
sys.path.append('..')
from utils import read_ising_file

model.eval()
temp_data = defaultdict(lambda: {'preds': [], 'labels': []})
folder = '../data/'

# Procesar archivos y hacer predicciones por lotes
for file in os.listdir(folder):
    if 'ising_' not in file:
        continue
    
    try:
        ising = read_ising_file(folder + file)
        temp = ising.metadata['T']
        label = ising.metadata['class']
        spins = ising.load_all_spins(invert=False)
        
        # Procesar todos los espines de un archivo en un solo batch
        with torch.no_grad():
            X = torch.tensor(spins, dtype=torch.float32).to(device)
            preds = (torch.sigmoid(model(X)) >= 0.5).long().cpu().numpy()
            
            temp_data[temp]['preds'].extend(preds)
            temp_data[temp]['labels'].extend([label] * len(preds))
    except (ValueError, KeyError, IndexError):
        continue

# Calcular accuracy por temperatura
temperaturas, accuracies = [], []
for temp in sorted(temp_data.keys()):
    preds = np.array(temp_data[temp]['preds'])
    labels = np.array(temp_data[temp]['labels'])
    accuracies.append(np.mean(preds == labels))
    temperaturas.append(temp)

temperaturas, accuracies = np.array(temperaturas), np.array(accuracies)

# Graficar
plt.figure(figsize=(10, 6))
plt.plot(temperaturas, accuracies, marker='o', linestyle='-', linewidth=2, markersize=8, color='#2E86AB')
plt.grid(True, linestyle='--', alpha=0.7)
plt.xlabel('Temperatura', fontsize=12)
plt.ylabel('Accuracy Promedio', fontsize=12)
plt.title('Accuracy del Modelo vs Temperatura', fontsize=14, fontweight='bold')
plt.ylim([0, 1.05])
plt.tight_layout()
plt.savefig('../img_results/accuracy_vs_temperature_nn.png', dpi=300, bbox_inches='tight')
plt.show()

# Mostrar resultados
print("\nAccuracy por temperatura:")
print("-" * 40)
for temp, acc in zip(temperaturas, accuracies):
    print(f"T = {temp:.3f}: {acc:.4f} ({acc*100:.2f}%)")

